## FEC 2018 Campaign Finance: Modeling

state

previously in politics

proportion individ contrib from in-state

party of prev officeholder 

primary results (is it open bc primary challenger beat incumbent)

force predict one winner per race

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from copy import deepcopy
from itertools import combinations 
from matplotlib import patheffects as path_effects
from sklearn import svm
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix,
)

%matplotlib inline

In [2]:
cand_orig = pd.read_csv('data/04bii_cand_cleaned.csv')

# include only house elections
cand_orig = cand_orig[cand_orig['position'] == 'H']
print(len(cand_orig))

# exclude races where Third party won
cand_orig = cand_orig[cand_orig['contest'] != 'MP_00']
print(len(cand_orig))

# exclude Third Party candidates for modeling for now
cand_orig = cand_orig[cand_orig['cand_pty_affiliation'] != 'Third party']
print(len(cand_orig))

# exclude open seat races; model requires one incumbent and one challenger
cand_orig = cand_orig[cand_orig['cand_ici'] != 'O']
print(len(cand_orig))

# exclude solo candidates
solos = list(cand_orig['contest'].value_counts()[cand_orig['contest'].value_counts() == 1].index)
for which in solos:
    cand_orig = cand_orig[cand_orig['contest'] != which]
print(len(cand_orig))

# exclude races with more than one candidate from each party
drops = []
for race in list(cand_orig['contest'].value_counts().index):
    lil_cand = cand_orig[cand_orig['contest'] == race]
    orig = len(lil_cand)
    lil_cand.drop_duplicates(['cand_pty_affiliation'], inplace = True)
    new = len(lil_cand)
    if orig != new:
        drops.append(race)
        print(race)
        
mask = [False if cand_orig.loc[i, 'contest'] in drops else True for i in list(cand_orig.index)]
cand_orig = cand_orig[mask]
print(len(cand_orig))

# exclude races with more than one incumbent (i.e. gerrymandering cases)
drops = []
for race in list(cand_orig['contest'].value_counts().index):
    lil_cand = cand_orig[cand_orig['contest'] == race]
    orig = len(lil_cand)
    lil_cand.drop_duplicates(['cand_ici'], inplace = True)
    new = len(lil_cand)
    if orig != new:
        drops.append(race)
        print(race)
        
mask = [False if cand_orig.loc[i, 'contest'] in drops else True for i in list(cand_orig.index)]
cand_orig = cand_orig[mask]
print(len(cand_orig))
    
cand_orig.head()

854
852
794
673
592
LA_03
LA_06
LA_01
CA_08
WI_04
WA_09
CA_27
CA_44
PA_03
568
PA_17
566


,cand_id,cand_name,cand_ici,cand_pty_affiliation,ttl_receipts,trans_from_auth,ttl_disb,trans_to_auth,coh_bop,coh_cop,...,comm2__TEXAS FARM BUREAU (in-kind),comm2__THE OORBEEK GROUP (in-kind),comm2__THE OORBEEK GROUP (nonaffiliated),comm2__THE WESTERVELT COMPANY PAC (nonaffiliated),comm2__TWICE-BAKED BISTRO BIS (in-kind),comm2__UNITED AIRLINES (in-kind),comm2__UNITED PARCEL SERVICE (in-kind),comm2__UNITED PARCEL SERVICE (nonaffiliated),comm2__WASHINGTON NATIONALS (in-kind),comm2__WINDOWS CATERING COMPANY (in-kind)
0,H0AL02087,"ROBY, MARTHA",I,Republican,2573681.12,85734.19,2277448.83,0.0,45450.72,341683.01,...,0.0,0.0,0.0,2500.0,0.0,0.0,0.0,0.0,0.0,0.0
1,H0AL05163,"BROOKS, MO",I,Republican,1527246.16,0.00,2006001.97,0.0,1170937.70,692181.51,...,0.0,0.0,0.0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0
3,H0AR01083,"CRAWFORD, ERIC ALAN RICK",I,Republican,1019464.16,0.00,966616.64,0.0,258577.00,311424.52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,H0AR03055,"WOMACK, STEVE",I,Republican,1211826.66,0.00,1218424.03,0.0,1165816.16,1159218.79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,H0AZ01259,"GOSAR, PAUL DR.",I,Republican,588764.42,16108.36,575157.87,0.0,107593.03,121199.58,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
cand_orig['type'].value_counts()

Group 01    266
Group 02    184
Group 04    114
Group 10      2
Name: type, dtype: int64

In [4]:
# flag each contest winner
# winner set constructed by manually reviewing each contest in Ballotpedia 
winner_cand = pd.read_csv('data/winners_2018.csv')
winner_dict = dict(zip(winner_cand['contest'], winner_cand['cand_id']))

cand_orig['winner_flag'] = [1 if cand_orig.loc[i, 'cand_id'] in list(winner_dict.values()) else \
                     0 for i in list(cand_orig.index)]

set(cand_orig['winner_flag'])

{0, 1}

In [ ]:
cand = cand_orig.drop([
    'cand_id',
    'cand_name',
    'cvg_end_dt',
#     'cand_class', 
#     'contest', # keep for join later
    'cand_loans', 
    'other_loans',
    'cand_office_district',
    'cand_office_st',
    'ttl_disb',
    'coh_cop',
    'cand_loan_repay',
    'other_loan_repay',
    'indiv_refunds',
    'cmte_refunds',
    'trans_to_auth',
    'debts_owed_by', # include? or time machine issue
    'position',
    'type',
], axis = 1)

if len(cand.isnull().sum()[cand.isnull().sum() > 0].sort_values()) == 0:
    print('no nulls')
else:
    print('check nulls')
    
cand.head()

In [ ]:
dummify = [
    'cand_ici',
    'cand_pty_affiliation',
#     'position'
]

for col in dummify:
    cand = pd.concat([
        cand.drop([col], axis = 1), 
        pd.get_dummies(cand[col])
    ], axis = 1)
    
cand.rename(columns={
    'C' : 'challenger',
    'I' : 'incumbent',
    'O' : 'open seat',
#     'H' : 'House',
#     'S' : 'Senate',
}, inplace = True)

In [ ]:
republicans = cand[cand['Republican'] == 1]
republicans.columns = [x + ' (R)' if x != 'contest' else x for x in republicans.columns]
republicans.set_index('contest', inplace = True)

democrats = cand[cand['Republican'] == 0]
democrats.columns = [x + ' (D)' if x != 'contest' else x for x in democrats.columns]
democrats.set_index('contest', inplace = True)

pre_cands = republicans.join(democrats, on = 'contest')
pre_cands.reset_index(drop = False, inplace = True)

pre_cands.head(2)

In [ ]:
pre_cands['r:d_funding_ratio'] = pre_cands['ttl_receipts (R)']/pre_cands['ttl_receipts (D)']

In [ ]:
contin_cols = [
    'ttl_receipts (R)',
    'trans_from_auth (R)',
    'coh_bop (R)',
    'cand_contrib (R)',
    'ttl_indiv_contrib (R)',
    'other_pol_cmte_contrib (R)',
    'pol_pty_contrib (R)',
    '(for) sum (R)',
    '(for) sum (D)',
    '(for) count (R)',
    '(for) count (D)',
    '(against) sum (R)',
    '(against) sum (D)',
    '(against) count (R)',
    '(against) count (D)',
    '(nonaffiliated) sum (R)',
    '(nonaffiliated) sum (D)',
    '(nonaffiliated) count (R)',
    '(nonaffiliated) count (D)',
    '(in-kind) sum (R)',
    '(in-kind) sum (D)',
    '(in-kind) count (R)',
    '(in-kind) count (D)',
    '(coord pty exp) sum (R)',
    '(coord pty exp) sum (D)',
    '(coord pty exp) count (R)',
    '(coord pty exp) count (D)',
    'ttl_receipts (D)',
    'trans_from_auth (D)',
    'coh_bop (D)',
    'cand_contrib (D)',
    'ttl_indiv_contrib (D)',
    'other_pol_cmte_contrib (D)',
    'pol_pty_contrib (D)',
    'r:d_funding_ratio',
    
]

contin = pre_cands[contin_cols]

contin_s = pd.DataFrame(
    MinMaxScaler().fit_transform(contin), 
    index = pre_cands.index, 
    columns = contin_cols,
)

the_rest = [x for x in pre_cands.columns if x not in contin_s.columns]

cands = pd.concat([contin_s, pre_cands[the_rest]], axis = 1)

cands.columns = [x.replace('Republican', 'republican').replace('Democrat', 'democrat') for x in cands.columns]

cands = cands[sorted(cands.columns)]

In [ ]:
cands.drop([
    'challenger (D)',
    'challenger (R)',
    'contest',
    'democrat (D)',
    'democrat (R)',
    'incumbent (D)',
#     'maxfunding_flag (D)',
#     'maxfunding_flag (R)',
    'republican (D)',
    'republican (R)',
    'winner_flag (D)',
    
], axis = 1, inplace = True)

In [ ]:
# partisan donors -- only gave to one party (listed columns are all 0s)
nones = list(cands.sum()[cands.sum() == 0].index)
nones = [x.split('__')[1] for x in nones]
list(nones)

In [ ]:
drops = cands.sum()[cands.sum() == 0].index

for which in drops:
    cands.drop(which, axis = 1, inplace = True)

In [ ]:
plt.figure(figsize = (16, 16))
corr_mat = np.tril(cands.corr(), k = -1)
ticks = [x.split('__')[1] if ('__' in x) else x for x in cands.columns]
sns.heatmap(corr_mat, cmap = 'PRGn', vmin = -1, vmax = 1)
plt.title('2018 U.S. House of Representatives elections:\nFeature correlation', fontsize = 14)
plt.xticks(
    np.arange(0.5, len(cands.columns), 1), 
    ticks, 
    fontsize = 7, 
    rotation = 90
)
plt.yticks(
    np.arange(0.5, len(cands.columns), 1), 
    ticks, 
    fontsize = 7, 
    rotation = 0
)
plt.xlim([0, len(cands.columns)])
plt.ylim([len(cands.columns), 0])
plt.tight_layout()
plt.show();

In [ ]:
X = deepcopy(cands)
X.drop(['winner_flag (R)'], axis = 1, inplace = True)

# target is whether incumbent wins
y = cands[['winner_flag (R)']]

X_train1, X_test1, y_train1, y_test1 = train_test_split(
    X[y['winner_flag (R)'] == 1], 
    y[y['winner_flag (R)'] == 1], 
    random_state = 421, 
    test_size = 0.33
)

X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X[y['winner_flag (R)'] == 0], 
    y[y['winner_flag (R)'] == 0], 
    random_state = 421, 
    test_size = 0.33
)

X_train = pd.DataFrame()
X_test = pd.DataFrame()
y_train = pd.DataFrame()
y_test = pd.DataFrame()

all_four = [
    [X_train1, X_train2],
    [X_test1, X_test2],
    [y_train1, y_train2],
    [y_test1, y_test2],
]

for i in range(2):
    X_train = pd.concat([X_train, all_four[0][i]], axis = 0)
    X_test = pd.concat([X_test, all_four[1][i]], axis = 0)
    y_train = pd.concat([y_train, all_four[2][i]], axis = 0)
    y_test = pd.concat([y_test, all_four[3][i]], axis = 0)

In [ ]:
def plot_confusion_matrix(
    confmat, 
    title = 'Confusion Matrix', 
    labels = ['Win', 'Loss'], 
    cmap = plt.cm.Blues,
):

    plt.figure(figsize = (10, 6))
    plt.imshow(
        confmat, 
        interpolation = 'nearest', 
        cmap = cmap, 
    )

    plt.grid(b = False)

    width, height = confmat.shape
    
    tick_marks = np.arange(width)
    plt.xticks(tick_marks, labels, fontsize = 14)
    plt.yticks(tick_marks, labels, fontsize = 14)
    plt.scatter(0, 0, color = (0,0,0,0))

    plt.title(title, fontsize = 14)
    plt.ylabel('Actual', fontsize = 14)
    plt.xlabel('Prediction', fontsize = 14)
    plt.tight_layout()

    for x in range(width):
        for y in range(height):
            plt.annotate(
                str(confmat[x][y]), 
                xy = (y, x), 
                horizontalalignment = 'center', 
                verticalalignment = 'center', 
                color = 'white', 
                fontsize = 40,
            ).set_path_effects(
                [
                    path_effects.Stroke(linewidth = 1, 
                                        foreground = 'black'
                                       ), 
                    path_effects.Normal()
                ]
            )
    return

### Naïve model

In [ ]:
y_pred_inc = [1 if (cands.loc[i, 'incumbent (R)'] == 1) else 0 for i in list(cands.index)]

In [ ]:
plot_confusion_matrix(
    confusion_matrix(cands['winner_flag (R)'], y_pred_inc),
    title = '2018 U.S. House of Representatives elections:\n\
    Naïve model confusion matrix\n--> choose incumbent',
)

In [ ]:
inc_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred_inc), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

inc_wrong = inc_y[inc_y['test'] != inc_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[inc_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

In [ ]:
y_pred_fund = [1 if (pre_cands.loc[i, 'r:d_funding_ratio'] > 1) else 0 for i in list(cands.index)]

In [ ]:
plot_confusion_matrix(
    confusion_matrix(cands['winner_flag (R)'], y_pred_fund),
    title = '2018 U.S. House of Representatives elections:\n\
    Naïve model confusion matrix\n--> choose higher funding',
)

In [ ]:
fund_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred_fund), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

fund_wrong = fund_y[fund_y['test'] != fund_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[fund_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

### Logistic regression

In [ ]:
lr = LogisticRegressionCV(cv = 11, random_state = 421)

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
lr.score(X_test, y_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_test, y_pred),
    title = '2018 U.S. House of Representatives elections:\n\
    Logistic regression confusion matrix',
)

In [ ]:
lr_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

lr_wrong = lr_y[lr_y['test'] != lr_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[lr_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

In [ ]:
coefs = pd.DataFrame(zip(X, lr.coef_[0]), columns = ['col', 'coef'])
coefs['abs_coef'] = [abs(x) for x in coefs['coef']]

coefs.sort_values(['abs_coef'], ascending = False, inplace = True)
coefs['color'] = (coefs['coef'] == coefs['abs_coef'])
coefs['color'] = ['green' if x == True else 'purple' for x in coefs['color']]
coefs.drop(['abs_coef'], axis = 1, inplace = True)
coefs = coefs[::-1]
coefs.reset_index(drop = True, inplace = True)

In [ ]:
plt.figure(figsize = (12, 24))
plt.barh(
    coefs.index, 
    coefs['coef'].apply(lambda x: abs(x)), 
    color = coefs['color'], 
    alpha = 0.8
)

plt.title('2018 U.S. House of Representatives elections:\n\
Logistic regression feature importance', fontsize = 14)
plt.ylabel('Feature', fontsize = 12)
plt.yticks(range(len(coefs)), coefs['col'].values)
plt.xlabel('Coefficient', fontsize = 12)
# plt.xticks()
plt.tight_layout()
plt.show();

### Random forest

In [ ]:
rfc = RandomForestClassifier(random_state=421)

rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
rfc.score(X_test, y_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_test, y_pred),
    title = '2018 U.S. House of Representatives elections:\n\
    Random forest confusion matrix',
)

In [ ]:
rfc_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

rfc_wrong = rfc_y[rfc_y['test'] != rfc_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[rfc_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

In [ ]:
coefs = pd.DataFrame(zip(X, rfc.feature_importances_), columns = ['col', 'coef'])
coefs['abs_coef'] = [abs(x) for x in coefs['coef']]

coefs.sort_values(['abs_coef'], ascending = False, inplace = True)
coefs['color'] = (coefs['coef'] == coefs['abs_coef'])
coefs['color'] = ['green' if x == True else 'purple' for x in coefs['color']]
coefs.drop(['abs_coef'], axis = 1, inplace = True)
coefs = coefs[::-1]
coefs.reset_index(drop = True, inplace = True)

In [ ]:
plt.figure(figsize = (8, 24))
plt.barh(
    coefs.index, 
    coefs['coef'].apply(lambda x: abs(x)), 
    color = coefs['color'], 
    alpha = 0.8
)

plt.title('2018 House of Representatives elections:\n\
Random forest feature importance', fontsize = 14)
plt.ylabel('Feature', fontsize = 12)
plt.yticks(range(len(coefs)), coefs['col'].values)
plt.xlabel('Importance', fontsize = 12)
plt.xticks(rotation = -45)
plt.tight_layout()
plt.show();

### Support vector machines

In [ ]:
SVM = svm.LinearSVC(random_state = 421)
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
SVM.score(X_test,y_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_test, y_pred),
    title = '2018 U.S. House of Representatives elections:\nSVM confusion matrix',
    labels = ['Win', 'Loss']
)

In [ ]:
svm_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

svm_wrong = svm_y[svm_y['test'] != svm_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[svm_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

### Neural network

In [ ]:
NN = MLPClassifier()

NN.fit(X_train, y_train)
y_pred = NN.predict(X_test)
NN.score(X_test, y_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_test, y_pred),
    title = '2018 U.S. House of Representatives elections:\nNN confusion matrix',
    labels = ['Win', 'Loss']
)

In [ ]:
nn_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

nn_wrong = nn_y[nn_y['test'] != nn_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[nn_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

## PCA

In [ ]:
from sklearn.decomposition import PCA

candspc = cands.drop(['winner_flag (R)'], axis = 1)

pca = PCA(n_components = len(candspc.columns))

pc = pca.fit(candspc)
# pccand = pd.DataFrame(
#     data = pc, 
# #     columns = ['principal component 1', 'principal component 2'], 
# )

In [ ]:
plt.bar(range(len(candspc.columns)), pc.explained_variance_)

In [ ]:
Xpc = deepcopy(candspc)

# target is whether incumbent wins
ypc = cands[['winner_flag (R)']]

Xpc_train1, Xpc_test1, ypc_train1, ypc_test1 = train_test_split(
    Xpc[ypc['winner_flag (R)'] == 1], 
    ypc[ypc['winner_flag (R)'] == 1], 
    random_state = 421, 
    test_size = 0.33
)

Xpc_train2, Xpc_test2, ypc_train2, ypc_test2 = train_test_split(
    Xpc[ypc['winner_flag (R)'] == 0], 
    ypc[ypc['winner_flag (R)'] == 0], 
    random_state = 421, 
    test_size = 0.33
)

Xpc_train = pd.DataFrame()
Xpc_test = pd.DataFrame()
ypc_train = pd.DataFrame()
ypc_test = pd.DataFrame()

all_four = [
    [Xpc_train1, Xpc_train2],
    [Xpc_test1, Xpc_test2],
    [ypc_train1, ypc_train2],
    [ypc_test1, ypc_test2],
]

for i in range(2):
    Xpc_train = pd.concat([Xpc_train, all_four[0][i]], axis = 0)
    Xpc_test = pd.concat([Xpc_test, all_four[1][i]], axis = 0)
    ypc_train = pd.concat([ypc_train, all_four[2][i]], axis = 0)
    ypc_test = pd.concat([ypc_test, all_four[3][i]], axis = 0)

### Logistic Regression: PCA

In [ ]:
lrpc = LogisticRegressionCV(cv = 11, random_state = 421)

lrpc.fit(Xpc_train, ypc_train)
y_pred = lrpc.predict(Xpc_test)
lrpc.score(Xpc_test, ypc_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_test, y_pred),
    title = '2018 U.S. House of Representatives elections:\n\
    Logistic regression confusion matrix',
)

In [ ]:
lr_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

lr_wrong = lr_y[lr_y['test'] != lr_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[lr_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

In [ ]:
coefs = pd.DataFrame(zip(Xpc, lrpc.coef_[0]), columns = ['col', 'coef'])
coefs['abs_coef'] = [abs(x) for x in coefs['coef']]

coefs.sort_values(['abs_coef'], ascending = False, inplace = True)
coefs['color'] = (coefs['coef'] == coefs['abs_coef'])
coefs['color'] = ['green' if x == True else 'purple' for x in coefs['color']]
coefs.drop(['abs_coef'], axis = 1, inplace = True)
coefs = coefs[::-1]
coefs.reset_index(drop = True, inplace = True)

In [ ]:
plt.figure(figsize = (12, 24))
plt.barh(
    coefs.index, 
    coefs['coef'].apply(lambda x: abs(x)), 
    color = coefs['color'], 
    alpha = 0.8
)

plt.title('2018 U.S. House of Representatives elections:\n\
Logistic regression feature importance', fontsize = 14)
plt.ylabel('Feature', fontsize = 12)
plt.yticks(range(len(coefs)), coefs['col'].values)
plt.xlabel('Coefficient', fontsize = 12)
# plt.xticks()
plt.tight_layout()
plt.show();

### Random forest

In [ ]:
rfcpc = RandomForestClassifier(random_state=421)

rfcpc.fit(Xpc_train, ypc_train)
ypc_pred = rfcpc.predict(Xpc_test)
rfcpc.score(Xpc_test, ypc_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(ypc_test, ypc_pred),
    title = '2018 U.S. House of Representatives elections:\n\
    Random forest confusion matrix',
)

In [ ]:
rfcpc_y = pd.DataFrame(
    zip(ypc_test['winner_flag (R)'], ypc_pred), 
    index = ypc_test.index, 
    columns = ['test', 'pred'],
)

rfcpc_wrong = rfcpc_y[rfcpc_y['test'] != rfcpc_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[rfcpc_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

In [ ]:
coefs = pd.DataFrame(zip(Xpc, rfcpc.feature_importances_), columns = ['col', 'coef'])
coefs['abs_coef'] = [abs(x) for x in coefs['coef']]

coefs.sort_values(['abs_coef'], ascending = False, inplace = True)
coefs['color'] = (coefs['coef'] == coefs['abs_coef'])
coefs['color'] = ['green' if x == True else 'purple' for x in coefs['color']]
coefs.drop(['abs_coef'], axis = 1, inplace = True)
coefs = coefs[::-1]
coefs.reset_index(drop = True, inplace = True)

In [ ]:
plt.figure(figsize = (8, 24))
plt.barh(
    coefs.index, 
    coefs['coef'].apply(lambda x: abs(x)), 
    color = coefs['color'], 
    alpha = 0.8
)

plt.title('2018 House of Representatives elections:\n\
Random forest feature importance', fontsize = 14)
plt.ylabel('Feature', fontsize = 12)
plt.yticks(range(len(coefs)), coefs['col'].values)
plt.xlabel('Importance', fontsize = 12)
plt.xticks(rotation = -45)
plt.tight_layout()
plt.show();

### Support vector machines

In [ ]:
SVMpc = svm.LinearSVC(random_state = 421)
SVMpc.fit(Xpc_train, ypc_train)
ypc_pred = SVMpc.predict(Xpc_test)
SVMpc.score(Xpc_test,ypc_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_test, y_pred),
    title = '2018 U.S. House of Representatives elections:\nSVM confusion matrix',
    labels = ['Win', 'Loss']
)

In [ ]:
svm_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

svm_wrong = svm_y[svm_y['test'] != svm_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[svm_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

### Neural network

In [ ]:
NN = MLPClassifier()

NN.fit(X_train, y_train)
y_pred = NN.predict(X_test)
NN.score(X_test, y_test)

In [ ]:
plot_confusion_matrix(
    confusion_matrix(y_test, y_pred),
    title = '2018 U.S. House of Representatives elections:\nNN confusion matrix',
    labels = ['Win', 'Loss']
)

In [ ]:
nn_y = pd.DataFrame(
    zip(y_test['winner_flag (R)'], y_pred), 
    index = y_test.index, 
    columns = ['test', 'pred'],
)

nn_wrong = nn_y[nn_y['test'] != nn_y['pred']]

pre_cands[[
    'contest', 
    'ttl_receipts (R)',
    'ttl_receipts (D)',
    'winner_flag (R)'
]].iloc[nn_wrong.index].sort_values(['ttl_receipts (R)'], ascending = False)

### Scatterplots

In [ ]:
marker_dict = {
    'I' : 'o',
    'C' : 'D',
    'O' : '+',
}

color_dict = {
    'Republican' : '#FF6661',
    'Democrat' : '#5494F7',
#     'Third party' : '#15DCDC',
}

alpha_dict = {
    1 : 1.0,
    0 : 0.2,
}

cand_orig['marker'] = [marker_dict[cand_orig.loc[i, 'cand_ici']] for i in list(cand_orig.index)]
cand_orig['color'] = [color_dict[cand_orig.loc[i, 'cand_pty_affiliation']] for \
                    i in list(cand_orig.index)]
cand_orig['alpha'] = [alpha_dict[cand_orig.loc[i, 'winner_flag']] for \
                    i in list(cand_orig.index)]

In [ ]:
orig_contin = list(set([x.replace(' (R)', '').replace(' (D)', '') for x in contin.columns]))
orig_contin = [x for x in orig_contin if x != 'r:d_funding_ratio']
for i in range(len(orig_contin)):
    for j in range(i + 1, len(orig_contin)):
        start_cand = deepcopy(cand_orig)
        for k in [i, j]:
            median = start_cand[orig_contin[k]].median()
            std = start_cand[orig_contin[k]].std()
            start_cand = start_cand[
                (start_cand[orig_contin[k]] > (median - 3*std)) & \
                (start_cand[orig_contin[k]] < (median + 3*std))
            ]
            start_cand = start_cand[start_cand[orig_contin[k]] > 0]
        if len(start_cand) > 1:
            plt.figure(figsize = (3, 3))
            for ici in start_cand['cand_ici'].value_counts().index:
                lil_cand = start_cand[start_cand['cand_ici'] == ici]
                marker = marker_dict[ici]
                for status in start_cand['winner_flag'].value_counts().index:
                    liller_cand = lil_cand[lil_cand['winner_flag'] == status]
                    alpha = alpha_dict[status]
                    plt.scatter(
                        liller_cand[orig_contin[i]], 
                        liller_cand[orig_contin[j]],
                        color = liller_cand['color'],
                        alpha = alpha,
                        marker = marker,
                        s = 30,
                    )
            plt.xlabel(orig_contin[i], fontsize = 12)
            plt.xticks([])
            plt.ylabel(orig_contin[j], fontsize = 12)
            plt.yticks([])

            plt.tight_layout()
            plt.show();

In [ ]:
# do ratio features
# what are repeating pol_pty_contrib values